# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

Lets have a look at the data:

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


It would first be useful to observe the call/no call ratio as a function of race:

In [4]:
call_ratio = pd.pivot_table(data, index='call', columns='race', values='id',aggfunc='count')
call_ratio

race,b,w
call,,
0.0,2278,2200
1.0,157,235


**1.** Conveniently Fisher test is perfect for this kind of 2x2 contingency tables <br>
https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.chi2_contingency.html<br>
https://en.wikipedia.org/wiki/G-test

The CLT applies for the mean call rate

**2.** $H_0$ = the callback ratio is identical for black and white sounding names

In [5]:
_, p_value, _, _ = stats.chi2_contingency(call_ratio, lambda_="log-likelihood")
print('p-value: {:1.3e}'.format(p_value))

p-value: 4.744e-05


The p-value is extremely low, so we will reject the null hypothesis, and conclude that for this sample there was a difference in the call rate for white and black sounding names

**3.** Margin of error for the mean  callback rate difference:

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

rate_w = np.sum(w.call)/len(w)
rate_b = np.sum(b.call)/len(b)
callback_rate = (np.sum(w.call) + np.sum(b.call))/(len(w) + len(b))
print('Callback rate for the population: {:1.3e}'.format(callback_rate))
#95% margin of error, assuming noral ditribution
callback_diff = rate_w - rate_b
p_h = (np.sum(w.call) + np.sum(b.call)) / (len(w) + len(b))
margin_of_error = 1.96*np.sqrt((p_h*(1-p_h)/(len(w) + len(b))))
print('margin of error (95% confidance): {:1.3e}'.format(margin_of_error))
conf_int = [callback_diff - margin_of_error, callback_diff + margin_of_error]
print('95% confidance interval {:1.3e}  to {:1.3e}'.format(conf_int[0], conf_int[1]))

Callback rate for the population: 8.049e-02
margin of error (95% confidance): 7.641e-03
95% confidance interval 2.439e-02  to 3.967e-02


And a bootsrap approach:

In [7]:
exp_num = 1000
bs_diff = np.empty(exp_num)
for i in range(exp_num):
    bs_w = np.random.choice(w.call, len(w))
    bs_b = np.random.choice(b.call, len(w))
    bs_rate_w = np.sum(bs_w)/len(bs_w)
    bs_rate_b = np.sum(bs_b)/len(bs_b)
    bs_diff[i] = bs_rate_w - bs_rate_b
    
conf_int = np.percentile(bs_diff, [2.75, 97.5])
print('95% confidance interval {:1.3e}  to {:1.3e}'.format(conf_int[0], conf_int[1]))

95% confidance interval 1.826e-02  to 4.723e-02


**4.** In the research we study the effect of the racial inference of a name on the chance getting  request for job interview. Fisher's exact test reveals a significant difference in the callback rates between black and white sounding names (in favor of the white sounding names). We have used a bootstrapping method to achieve a margin of error on the difference in callback rate. While the general sample population achieves a callback rate of 8% there is a 1.8%-4.7% advantage to white sounding names (which is at least 50% difference in favor of white sounding names).

**5.** While the research suggests that racial discrimination exists, we have not explored all the relevant variables. If we trust the researchers and all other variables were the same, this is actually an evidence for racial discrimination.